In [30]:
import illustris_python as il
import requests
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import os

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
baseUrl = 'http://www.tng-project.org/api/TNG-Cluster'
headers = {"api-key":os.getenv('API_KEY')}
base_dir = 'tng_cache/'

def get(path, params=None, save_dir=''):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = base_dir + save_dir + r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [33]:
sim = get(baseUrl)
sim

{'name': 'TNG-Cluster',
 'description': 'Main high-resolution TNG-Cluster simulation suite (virtual box), with the full fiducial TNG physics model.',
 'name_alt': 'L680n8192TNG',
 'boxsize': 680000.0,
 'z_start': 127.0,
 'z_final': 0.0,
 'cosmology': 'Planck2015',
 'omega_0': 0.3089,
 'omega_L': 0.6911,
 'omega_B': 0.0486,
 'hubble': 0.6774,
 'physics_model': 'Fiducial TNG Model',
 'has_cooling': True,
 'has_starformation': True,
 'has_winds': True,
 'has_blackholes': True,
 'mass_gas': 0.000771495,
 'mass_dm': 0.00413211373831018,
 'softening_dm_comoving': 2.0,
 'softening_stars_comoving': 2.0,
 'softening_blackholes_comoving': 4.0,
 'softening_gas_comoving': 2.0,
 'softening_dm_max_phys': 1.0,
 'softening_stars_max_phys': 1.0,
 'softening_blackholes_max_phys': 4.0,
 'softening_gas_max_phys': 1.0,
 'softening_gas_factor': 2.5,
 'softening_gas_comoving_min': 0.25,
 'num_dm': 549755813888,
 'num_tr_mc': 15416383432,
 'num_tr_vel': 0,
 'longids': False,
 'is_uniform': True,
 'is_zoom': F

In [90]:
# get z = 0 snapshot
snap_index = 99
snap_dir = f'snap_{snap_index:03d}/'
snap_dir

'snap_099/'

In [19]:
offsets = get(sim['files']['offsets'])
offset = get(offsets['files'][snap_index], save_dir=snap_dir)
offset

{'files': ['http://www.tng-project.org/api/TNG-Cluster/files/offsets.0.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.1.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.2.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.3.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.4.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.5.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.6.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.7.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.8.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.9.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.10.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.11.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.12.hdf5',
  'http://www.tng-project.org/api/TNG-Cluster/files/offsets.13.hdf5',
  'http://www.tng-pro

In [35]:
snaps = get(sim['snapshots'])
snap = get(snaps[snap_index]['url'], save_dir=snap_dir)
snap

{'simulation': 'http://www.tng-project.org/api/TNG-Cluster/',
 'number': 99,
 'redshift': 0.0,
 'num_gas': 14969664891,
 'num_dm': 14829431232,
 'num_trmc': 15416383432,
 'num_stars': 1067549783,
 'num_bhs': 430359,
 'num_groups_fof': 19453039,
 'num_groups_subfind': 10378451,
 'num_groups_rockstar': 0,
 'filesize_snapshot': 4928426793596.0,
 'filesize_groupcat': 16110992699.0,
 'filesize_rockstar': 0.0,
 'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/',
 'subhalos': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/',
 'vis': {'groupcat_sfms': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=ssfr&xlim=9.0%2C11.0',
  'groupcat_sizemass': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=size_stars',
  'groupcat_mz_gas': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=Z_gas',
  'groupcat_mz_stars': 'http

In [36]:
subs = get( snap['subhalos'], {'limit':50, 'order_by':'-mass_stars'})
subs

{'count': 10378451,
 'next': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/?limit=50&offset=50&order_by=-mass_stars',
 'previous': None,
 'results': [{'id': 4680430,
   'sfr': 6.98751,
   'mass_log_msun': 14.823941436818016,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/4680430/'},
  {'id': 2930523,
   'sfr': 0.100453,
   'mass_log_msun': 15.20857685911439,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/2930523/'},
  {'id': 845400,
   'sfr': 0.0373764,
   'mass_log_msun': 15.405578694454283,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/845400/'},
  {'id': 8143526,
   'sfr': 320.685,
   'mass_log_msun': 14.61170512428078,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/8143526/'},
  {'id': 6419490,
   'sfr': 1924.89,
   'mass_log_msun': 14.651030081787413,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6419490/'},
  {'id':

In [37]:
# get 50th most massive subhalo and its siblings
sub = get( subs['results'][-1]['url'] )
sub

{'snap': 99,
 'id': 6878279,
 'bhmdot': 0.0337376,
 'cm_x': 32693.5,
 'cm_y': 674148.0,
 'cm_z': 20570.1,
 'gasmetallicity': 0.0397298,
 'gasmetallicityhalfrad': 0.0472337,
 'gasmetallicitymaxrad': 0.0513365,
 'gasmetallicitysfr': 0.0469282,
 'gasmetallicitysfrweighted': 0.0465851,
 'pos_x': 32626.5,
 'pos_y': 674116.0,
 'pos_z': 20582.8,
 'halfmassrad': 364.721,
 'halfmassrad_gas': 436.339,
 'halfmassrad_dm': 695.093,
 'halfmassrad_stars': 10.659,
 'halfmassrad_bhs': 0.0,
 'len': 1670107,
 'len_gas': 982585,
 'len_dm': 604105,
 'len_stars': 28611,
 'len_bhs': 28,
 'mass': 20075.8,
 'mass_gas': 2477.86,
 'mass_dm': 2496.23,
 'mass_stars': 604.422,
 'mass_bhs': 3.53376,
 'massinhalfrad': 378.5,
 'massinhalfrad_gas': 2.52405,
 'massinhalfrad_dm': 0.095039,
 'massinhalfrad_stars': 302.177,
 'massinhalfrad_bhs': 3.35912,
 'massinmaxrad': 179.373,
 'massinmaxrad_gas': 0.710223,
 'massinmaxrad_dm': 0.008264,
 'massinmaxrad_stars': 165.775,
 'massinmaxrad_bhs': 3.35912,
 'massinrad': 607.005,

In [71]:
parent_request = { 'limit': '10000' }
parent = get(sub['related']['parent_halo'], params=parent_request )
parent_info = get(parent['meta']['info'])
parent_info

{'SimulationName': 'L680n8192TNG',
 'SnapshotNumber': 99,
 'InfoType': 'Group',
 'InfoID': 12925051,
 'GroupBHMass': 0.7707715034484863,
 'GroupBHMdot': 0.033797066658735275,
 'GroupCM': [32708.49609375, 674166.8125, 20553.513671875],
 'GroupContaminationFracByMass': 0.7769969701766968,
 'GroupContaminationFracByNumPart': 0.051520563662052155,
 'GroupFirstSub': 6878279,
 'GroupGasMetalFractions': [0.7514222264289856,
  0.24420961737632751,
  0.00048187043284997344,
  0.0001592925691511482,
  0.002136792754754424,
  0.0007315163384191692,
  0.00020899852097500116,
  0.00019661607802845538,
  0.0003065162745770067,
  0.00014658596774097532],
 'GroupGasMetallicity': 0.004368188790977001,
 'GroupLen': 2298752,
 'GroupLenType': [1000081, 1088475, 59125, 0, 151020, 51],
 'GroupMass': 23464.6640625,
 'GroupMassType': [2507.251708984375,
  4497.70263671875,
  15671.0908203125,
  0.0,
  784.6505737304688,
  3.968032121658325],
 'GroupNsubs': 399,
 'GroupOffsetType': [4007499630, 6349110121, 317

In [72]:
parent['child_subhalos']

{'count': 399,
 'next': None,
 'previous': None,
 'results': [{'id': 6878279,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878279/'},
  {'id': 6878280,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878280/'},
  {'id': 6878281,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878281/'},
  {'id': 6878282,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878282/'},
  {'id': 6878283,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878283/'},
  {'id': 6878284,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878284/'},
  {'id': 6878285,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878285/'},
  {'id': 6878286,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6878286/'},
  {'id': 6878287,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalo

In [73]:
sibling_subs = [ 
    get(res['url']) for res in parent['child_subhalos']['results']
]
# array entries
sub_keys = set(sibling_subs[0].keys()) - { 'related', 'cutouts', 'trees', 'supplementary_data', 'vis', 'meta' }
sub_keys


{'bhmdot',
 'cm_x',
 'cm_y',
 'cm_z',
 'desc_sfid',
 'desc_snap',
 'gasmetallicity',
 'gasmetallicityhalfrad',
 'gasmetallicitymaxrad',
 'gasmetallicitysfr',
 'gasmetallicitysfrweighted',
 'grnr',
 'halfmassrad',
 'halfmassrad_bhs',
 'halfmassrad_dm',
 'halfmassrad_gas',
 'halfmassrad_stars',
 'id',
 'len',
 'len_bhs',
 'len_dm',
 'len_gas',
 'len_stars',
 'mass',
 'mass_bhs',
 'mass_dm',
 'mass_gas',
 'mass_log_msun',
 'mass_stars',
 'massinhalfrad',
 'massinhalfrad_bhs',
 'massinhalfrad_dm',
 'massinhalfrad_gas',
 'massinhalfrad_stars',
 'massinmaxrad',
 'massinmaxrad_bhs',
 'massinmaxrad_dm',
 'massinmaxrad_gas',
 'massinmaxrad_stars',
 'massinrad',
 'massinrad_bhs',
 'massinrad_dm',
 'massinrad_gas',
 'massinrad_stars',
 'parent',
 'pos_x',
 'pos_y',
 'pos_z',
 'primary_flag',
 'prog_sfid',
 'prog_snap',
 'sfr',
 'sfrinhalfrad',
 'sfrinmaxrad',
 'sfrinrad',
 'snap',
 'spin_x',
 'spin_y',
 'spin_z',
 'starmetallicity',
 'starmetallicityhalfrad',
 'starmetallicitymaxrad',
 'stellarph

In [109]:
parent_request = {
    'gas':'Coordinates,Masses,ParticleIDs,Density,ElectronAbundance,GFM_Metals',
    'stars':'Coordinates,Masses,ParticleIDs,GFM_StellarFormationTime'
}
parent_cutout = get(sub['cutouts']['parent_halo'], parent_request, save_dir=snap_dir)


KeyboardInterrupt: 

In [117]:
sibling_request = {
    'gas':'ParticleIDs',
    'stars':'ParticleIDs'
}
for sub in sibling_subs:
    if sub['len_gas'] + s['len_stars'] > 0:
        get(sub['cutouts']['subhalo'], sibling_request, save_dir=snap_dir)

In [133]:
with h5py.File(base_dir + snap_dir + f'cutout_{parent_info["InfoID"]}.hdf5', 'r') as f:
    gas_ids = f['PartType0/ParticleIDs'][:]
    star_ids = f['PartType4/ParticleIDs'][:]

    gas_sub_ids = -np.ones_like(gas_ids, dtype=np.int64)
    star_sub_ids = -np.ones_like(star_ids, dtype=np.int64)

for sub in sibling_subs:
    if sub['len_gas'] + s['len_stars'] > 0:
        with h5py.File(base_dir + snap_dir + f'cutout_{sub["id"]}.hdf5', 'r') as sub_f:
            
            if sub['len_gas'] > 0:
                sub_gas_ids = sub_f['PartType0/ParticleIDs'][:]
                gas_sub_ids[np.isin(gas_ids, sub_gas_ids)] = sub["id"]
            
            if sub['len_stars'] > 0:
                sub_star_ids = sub_f['PartType4/ParticleIDs'][:]
                star_sub_ids[np.isin(star_ids, sub_star_ids)] = sub["id"]

with h5py.File(base_dir + snap_dir + f'cutout_{parent_info["InfoID"]}.hdf5', 'a') as f:
        f['PartType0/SubhaloIDs'] = gas_sub_ids
        f['PartType4/SubhaloIDs'] = star_sub_ids

In [134]:
with h5py.File("outputs/dm_proto.hdf5", "w") as f:
    g_halo = f.create_group(f'halo_{parent_info["InfoID"]}')
    g_halo['GroupCM'] = parent_info['GroupCM']
    g_halo['GroupMassType'] = parent_info['GroupMassType']
    g_halo['GroupSFR'] = parent_info['GroupSFR']
    g_halo['Group_R_Crit500'] = parent_info['Group_R_Crit500']
    g_subs = g_halo.create_group('subhalos')
    for k in sub_keys:
        g_subs[k] = [ s[k] for s in sibling_subs ]

In [135]:
with h5py.File("outputs/dm_proto.hdf5", "r") as f:
    print(f['halo_12925051']['subhalos'].keys())

<KeysViewHDF5 ['bhmdot', 'cm_x', 'cm_y', 'cm_z', 'desc_sfid', 'desc_snap', 'gasmetallicity', 'gasmetallicityhalfrad', 'gasmetallicitymaxrad', 'gasmetallicitysfr', 'gasmetallicitysfrweighted', 'grnr', 'halfmassrad', 'halfmassrad_bhs', 'halfmassrad_dm', 'halfmassrad_gas', 'halfmassrad_stars', 'id', 'len', 'len_bhs', 'len_dm', 'len_gas', 'len_stars', 'mass', 'mass_bhs', 'mass_dm', 'mass_gas', 'mass_log_msun', 'mass_stars', 'massinhalfrad', 'massinhalfrad_bhs', 'massinhalfrad_dm', 'massinhalfrad_gas', 'massinhalfrad_stars', 'massinmaxrad', 'massinmaxrad_bhs', 'massinmaxrad_dm', 'massinmaxrad_gas', 'massinmaxrad_stars', 'massinrad', 'massinrad_bhs', 'massinrad_dm', 'massinrad_gas', 'massinrad_stars', 'parent', 'pos_x', 'pos_y', 'pos_z', 'primary_flag', 'prog_sfid', 'prog_snap', 'sfr', 'sfrinhalfrad', 'sfrinmaxrad', 'sfrinrad', 'snap', 'spin_x', 'spin_y', 'spin_z', 'starmetallicity', 'starmetallicityhalfrad', 'starmetallicitymaxrad', 'stellarphotometrics_b', 'stellarphotometrics_g', 'stellar